In [285]:
#now we can start doing some machine learning analysis
#import required modules
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle

In [286]:
#unpack pickled dataframes

modeling = pd.read_pickle("modeling.pkl")
forecast = pd.read_pickle("forecast.pkl")

#change differential into win/loss
modeling["Differential"] = modeling["Differential"].apply(lambda x: 1 if x > 0 else 0)
modeling.head()

,Differential,QB FOR,WR1 FOR,WR2 FOR,WR3 FOR,TE FOR,HB FOR,DT FOR,LT FOR,LE FOR,...,FS AGAINST,SS AGAINST,CB1 AGAINST,CB2 AGAINST,RT AGAINST,C AGAINST,LG AGAINST,RG AGAINST,K AGAINST,P AGAINST
gamename,,,,,,,,,,,,,,,,,,,,,
Chiefs Texans 2020,1,99,98,81,78,99,84,92,75,73,...,83,72,85,78,70,70,67,68,72,75
Seahawks Falcons 2020,1,95,89,87,77,80,87,78,83,73,...,77,80,78,77,71,87,68,79,81,71
Bills Jets 2020,1,88,97,87,82,75,80,80,81,76,...,75,85,82,74,69,75,73,69,67,74
Raiders Panthers 2020,1,83,80,79,77,93,90,79,79,77,...,80,66,82,76,85,78,66,68,74,77
Bears Lions 2020,1,71,90,75,75,78,82,82,79,88,...,80,74,80,79,74,86,71,69,80,80


In [287]:
#split modeling data in to test and train (75/25)
X = modeling.drop("Differential", axis = 1)
Y = modeling["Differential"]

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state = 0)

In [288]:
#scaling the data
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X_train)
train_scaled = scaler.transform(X_train)
test_scaled = scaler.transform(X_test)
X_scaled1 = scaler.transform(X)

scaler2 = preprocessing.StandardScaler().fit(X)
X_scaled2 = scaler2.transform(X)
forecast_scaled = scaler2.transform(forecast)

In [289]:
#A logistic regression model gives nice results with both training and test data
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression().fit(train_scaled,Y_train)
print(np.sum(np.array(LR.predict(test_scaled)) == np.array(Y_test))/len(X_test))
print(np.sum(np.array(LR.predict(train_scaled)) == np.array(Y_train))/len(X_train))

0.6704545454545454
0.7146464646464646


In [290]:
#cross validating
#looks like the the model will be correct ~69% of the time on average
from sklearn.model_selection import cross_val_score
cvscore = cross_val_score(LR, X_scaled1, Y, cv = 10)
np.average(cvscore)

0.6854896675651393

In [293]:
#applying the model to forecast the 2022 season
LR = LogisticRegression().fit(X_scaled2,Y)
forecast["result"] = LR.predict(forecast_scaled)

In [295]:
forecast["result"] = forecast["result"].apply(lambda x: "W" if x > 0 else "L")
forecast["result"].to_frame().to_csv("2022predictions.csv")